# Analysis Notebook for the scraper project

In [1]:
import pandas as pd
import os
import numpy as np
from pathlib import Path
import os
import sys
import re

In [2]:
# prevent hiding of columns
pd.options.display.max_columns = None

# prevent cropping of lng strings for display
pd.options.display.max_colwidth = 300

In [3]:
# no wrapping of long strings

In [4]:
%%html
<style>
.dataframe td {
    white-space: nowrap;
}
</style>

In [5]:
root = Path(os.getcwd()).parent
fname = root / 'data' / 'processed_data.csv'
df=pd.read_csv(fname)
df

,Unnamed: 0,Unnamed: 0.1,index_url,Unnamed: 0.1.1,data-advert-position,class,href,data-attr-href,alt,data-trace,data-category,data-label,data-actions,fname,scrape_ts,processed,domain,category1,bid_ask,rent_buy,rooms,cost,address,duration,details,text,timestamp
0,0,0.0,https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wunderschoenes-co-living-haus-mit-38-suiten-in-kloten.2042131?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1,0.0,2042131_1220_17339d9c5389a0a775dc1d31c96d5c9b,[],https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wunderschoenes-co-living-haus-mit-38-suiten-in-kloten.2042131?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1,https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wunderschoenes-co-living-haus-mit-38-suiten-in-kloten.2042131?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1,"Wunderschönes Co-Living Haus mit 38 Suiten in Kloten,...",NaN,ContentList,ContentTitle,click,C:\coding\wgscraper\data\ronorp\adpage_2020-10-19_17-21-49_163922.scrape,2020-10-19 17:21:49.163922,False,ronorp,Wohnung / Zimmer,Biete,Mieten,1.0,1'095.00,"Marktgasse 19, 8038 Kloten",unbefristet,"Biete / Suche / Tausche: Wohnung / Zimmer: Biete Mieten & Kaufen: Mieten Stadt / Agglo: Agglo Vertragsart: unbefristet Zimmer: 1Kosten: 1'095.00Adresse: Marktgasse 19, 8038 Kloten Kontakt: i********@t********************h","Ich glaube, dass zusammen einfach schöner ist als alleine. Deshalb habe ich ein Start-up gegründet, das Gebäude für grosse Hausgemeinschaften baut.\nIn Kloten eröffnen wir nun am 1.Oktober unser erstes Gebäude mit 38 Suiten, komplett mit jeweils eigenem Bad (Dusche & WC). Die meisten Suiten habe...",2020-02-09 10:53:00
1,1,1.0,https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wg-zimmer-am-idaplatz-bis-ende-juni-2022.2074090?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1,1.0,2074090_1220_bf9d198c378aaf06153dd1319589670b,[],https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wg-zimmer-am-idaplatz-bis-ende-juni-2022.2074090?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1,https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wg-zimmer-am-idaplatz-bis-ende-juni-2022.2074090?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1,WG-Zimmer am Idaplatz bis Ende Juni 2022,NaN,ContentList,ContentTitle,click,C:\coding\wgscraper\data\ronorp\adpage_2020-10-19_17-21-51_717955.scrape,2020-10-19 17:21:51.717955,False,ronorp,Wohnung / Zimmer,Biete,Mieten,1.0,600.00,"Idaplatz 2, 8003 Zürich",befristet,"Biete / Suche / Tausche: Wohnung / Zimmer: Biete Mieten & Kaufen: Mieten Stadt / Agglo: Stadt Vertragsart: befristet (auf Zeit) Zimmer: 1Kosten: 600.00Adresse: Idaplatz 2, 8003 Zürich Kontakt: w*****@g***********m","Per Februar 2021 wird in unserer 3er WG direkt am Idaplatz ein Zimmer frei. Die 4-Zimmerwohnung befindet sich im 2ten Stock mit zwei Balkone, einen in den ruhigen Innenhof, der andere direkt über dem Idaplatz. Zudem gibt es ein Wohnzimmer, eine Geschirrspülmaschine, zwei Kühl- & Gefrierschränke ...",2020-10-19 17:17:00
2,2,2.0,https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/zimmer-in-wiedikon-ab-november.2073817?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1%26page%3D2%26hash%3D8066eefae06f4ebdde76511cbb811cdf,2.0,2073817_1220_fe72ecc8e4d34be149e7261f78f629cd,[],https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/zimmer-in-wiedikon-ab-november.2073817?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1%26page%3D2%26hash%3D8066eefae06f4ebdde76511cbb811cdf,https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/zimmer-in-wiedikon-ab-november.2073817?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1%26page%3D2%26hash%3D8066eefae06f4ebdde76511cbb811cdf,Zimmer in Wiedikon ab November,NaN,ContentList,ContentTitle,click,C:\coding\wgscraper\data\ronorp\adpage_2020-10-19_17-21-54_263640.scrape,2020-10-19 17:21:54.263640,False,ronorp,Wohnung / Zimmer,Biete,Mieten,1.0,850.00,"Üetlibergstrasse 139, 8

In [16]:

textpatterns=[
    r'mehralswohnen',
    r'dialogweg',
    r'kraftwerk',
    r'hochfoif',
    r'mehr als 11',
    r'mehrals11',
    r'd6s4-SUED',
    r'Hunzkomune',
    r'Bunter Haufen',
    r'BunterHaufen',
    r'BuntGemischt',
    r'Bunt-Gemischt',
    r'Brückenwohnen',
#     r'MIR',
    r'HUNZREAL',
    r'kalkbreite',
    r'hunziker',
    r'Mehrgeneration',
    r'kraftwerk1',
    r'suite55',
    r'suite37',
    
]

addresspatterns=[
    r'dialogweg',
    r'hunziker'
]




searchdict=[
    *[{'column':'text', 'pattern':pat,'flag':'IGNORECASE'} for pat in textpatterns],
    *[{'column':'address', 'pattern':pat,'flag':'IGNORECASE'} for pat in addresspatterns],
#     {'column':'address', 'pattern': r'dialogweg','flag':'IGNORECASE'},
#     {'column':'address', 'pattern': r'hunziker','flag':'IGNORECASE'},
    
    
    
]
searchdict

[{'column': 'text', 'pattern': 'mehralswohnen', 'flag': 'IGNORECASE'},
 {'column': 'text', 'pattern': 'dialogweg', 'flag': 'IGNORECASE'},
 {'column': 'text', 'pattern': 'kraftwerk', 'flag': 'IGNORECASE'},
 {'column': 'text', 'pattern': 'hochfoif', 'flag': 'IGNORECASE'},
 {'column': 'text', 'pattern': 'mehr als 11', 'flag': 'IGNORECASE'},
 {'column': 'text', 'pattern': 'mehrals11', 'flag': 'IGNORECASE'},
 {'column': 'text', 'pattern': 'd6s4-SUED', 'flag': 'IGNORECASE'},
 {'column': 'text', 'pattern': 'Hunzkomune', 'flag': 'IGNORECASE'},
 {'column': 'text', 'pattern': 'Bunter Haufen', 'flag': 'IGNORECASE'},
 {'column': 'text', 'pattern': 'BunterHaufen', 'flag': 'IGNORECASE'},
 {'column': 'text', 'pattern': 'BuntGemischt', 'flag': 'IGNORECASE'},
 {'column': 'text', 'pattern': 'Bunt-Gemischt', 'flag': 'IGNORECASE'},
 {'column': 'text', 'pattern': 'Brückenwohnen', 'flag': 'IGNORECASE'},
 {'column': 'text', 'pattern': 'HUNZREAL', 'flag': 'IGNORECASE'},
 {'column': 'text', 'pattern': 'kalkbre

In [21]:
results=[]
for search in searchdict:
    col=search['column']
    pat=search['pattern']
    if search['flag'] == 'IGNORECASE':
        results.append(df[df[col].str.contains(pat,flags=re.IGNORECASE, regex=True)] )
    else:
        results.append(df[df[col].str.contains(pat, regex=True)] )
        
results2=pd.concat(results).drop_duplicates(subset=['alt'])
showcols=['timestamp','domain', 'category1','bid_ask','rent_buy', 'rooms', 'cost', 'address','duration','alt','href']
results2[showcols]


,timestamp,domain,category1,bid_ask,rent_buy,rooms,cost,address,duration,alt,href
35,2020-10-15 16:22:00,ronorp,Wohnung / Zimmer,Biete,Mieten,5.5,590.00,"Dialogweg 3, 8001 Zürich",unbefristet,Room vacant in friendly shared household in a cooperative...,https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/room-vacant-in-friendly-shared-household-in-a-cooperative-in.2071402?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1%26page%3D13%26hash%3D8066eefae06f4ebdde76511cbb811cdf
41,2020-10-14 21:11:00,ronorp,Wohnung / Zimmer,Biete,Mieten,1.0,900.00,"Dialogweg 6, 8050 Zürich",unbefristet,Zimmer in Gross-WG auf dem Hunziker Areal per 1. Dezember...,https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/zimmer-in-gross-wg-auf-dem-hunziker-areal-per-1-dezember-zu.2070775?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1%26page%3D15%26hash%3D8066eefae06f4ebdde76511cbb811cdf
112,2020-04-10 17:11:00,ronorp,Wohnung / Zimmer,Biete,Mieten,1.0,952.00,"Kalkbreitestrasse 6, 8003 Zürich",unbefristet,Sechsköpfige Familien-WG mit zwei kleinen Kindern sucht...,https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/sechskoepfige-familien-wg-mit-zwei-kleinen-kindern-sucht.2064122?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1%26page%3D38%26hash%3D8066eefae06f4ebdde76511cbb811cdf


In [20]:
df[df['href'].str.contains('Hunz',case=False)]


,Unnamed: 0,index_url,Unnamed: 0.1,data-advert-position,class,href,data-attr-href,alt,data-trace,data-category,data-label,data-actions,fname,scrape_ts,processed,domain,category1,bid_ask,rent_buy,rooms,cost,address,duration,details,text,timestamp


In [7]:
fname='bluub..csv'
df2=df.set_index('index_url')
df2.to_csv(fname)
df2

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,alt,class,data-actions,data-advert-position,data-attr-href,data-category,data-label,data-trace,fname,href,processed,scrape_ts
index_url,,,,,,,,,,,,,,,
https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wunderschoenes-co-living-haus-mit-38-suiten-in-kloten.2042131?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1,0,0.0,0,"Wunderschönes Co-Living Haus mit 38 Suiten in Kloten,...",[],click,2042131_1220_cf282400a14da664718af86e91aa9d15,https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wunderschoenes-co-living-haus-mit-38-suiten-in-kloten.2042131?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1,ContentList,ContentTitle,NaN,C:\coding\wgscraper\data\ronorp\adpage_2020-10-18_22-18-47_969143.scrape,https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wunderschoenes-co-living-haus-mit-38-suiten-in-kloten.2042131?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1,False,2020-10-18 22:18:47.969143
NaN,1,1.0,https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wunderschoenes-co-living-haus-mit-38-suiten-in-kloten.2042131?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1,"Wunderschönes Co-Living Haus mit 38 Suiten in Kloten,...",[],click,2042131_1220_c007fee2e6ef5fef97b2b7b81a7c63bc,https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wunderschoenes-co-living-haus-mit-38-suiten-in-kloten.2042131?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1,ContentList,ContentTitle,NaN,C:\coding\wgscraper\data\ronorp\adpage_2020-10-19_00-09-27_570444.scrape,https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wunderschoenes-co-living-haus-mit-38-suiten-in-kloten.2042131?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1,False,2020-10-19 00:09:27.570444
NaN,2,2.0,https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/schoenes-studio-in-wollerau.2072370?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1%26sort%3Dcreated%26page%3D7%26hash%3D8066eefae06f4ebdde76511cbb811cdf,Schönes Studio in Wollerau,[],click,2072370_1220_3bd7300806c559e9a9c92bb64096ff30,https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/schoenes-studio-in-wollerau.2072370?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1%26sort%3Dcreated%26page%3D7%26hash%3D8066eefae06f4ebdde76511cbb811cdf,ContentList,ContentTitle,NaN,C:\coding\wgscraper\data\ronorp\adpage_2020-10-19_00-09-59_659456.scrape,https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/schoenes-studio-in-wollerau.2072370?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1%26sort%3Dcreated%26page%3D7%26hash%3D8066eefae06f4ebdde76511cbb811cdf,False,2020-10-19 00:09:59.659456
NaN,3,3.0,https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/3-5-zimmer-luxuswohnung-zu-vermieten-in-bilten.2072351?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1%26sort%3Dcreated%26page%3D7%26hash%3D8066eefae06f4ebdde76511cbb811cdf,"3,5 Zimmer Luxuswohnung zu vermieten in Bilten",[],click,2072351_1220_bb8d551ebe16be76d4bdb276fad4c24a,https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/3-5-zimmer-luxuswohnung-zu-vermieten-in-bilten.2072351?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1%26sort%3Dcreated%26page%3D7%26hash%3D8066eefae06f4ebdde76511cbb811cdf,ContentList,ContentTitle,NaN,C:\coding\wgscraper\data\ronorp\adpage_2020-10-19_00-10-05_825444.scrape,https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/3-5-zimmer-luxuswohnung-zu-vermieten-in-bilten.2072351?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1%26sort%3Dcreated%26page%3D7%26hash%3D8066eefae06f4ebdde76511cbb811cdf,False,2020-10-19 00:10:05.825444
NaN,4,4.0,https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/dachzimmer-in-wg-am-brupbacherplatz-wochenaufenthalter-in.2070785?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1%26page%3D13%26hash%3D8066eefae06f4ebdde76511cbb811cdf,Dachzimmer in WG am Brupbacherplatz //...,[],click,2070785_1220_55792fd7a5cf63254d44f7a2

In [12]:
df3=pd.read_csv(fname,index_col='index_url')
df3

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,alt,class,data-actions,data-advert-position,data-attr-href,data-category,data-label,data-trace,fname,href,processed,scrape_ts
index_url,,,,,,,,,,,,,,,
https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wunderschoenes-co-living-haus-mit-38-suiten-in-kloten.2042131?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1,0,0.0,0,"Wunderschönes Co-Living Haus mit 38 Suiten in Kloten,...",[],click,2042131_1220_cf282400a14da664718af86e91aa9d15,https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wunderschoenes-co-living-haus-mit-38-suiten-in-kloten.2042131?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1,ContentList,ContentTitle,NaN,C:\coding\wgscraper\data\ronorp\adpage_2020-10-18_22-18-47_969143.scrape,https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wunderschoenes-co-living-haus-mit-38-suiten-in-kloten.2042131?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1,False,2020-10-18 22:18:47.969143
NaN,1,1.0,https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wunderschoenes-co-living-haus-mit-38-suiten-in-kloten.2042131?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1,"Wunderschönes Co-Living Haus mit 38 Suiten in Kloten,...",[],click,2042131_1220_c007fee2e6ef5fef97b2b7b81a7c63bc,https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wunderschoenes-co-living-haus-mit-38-suiten-in-kloten.2042131?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1,ContentList,ContentTitle,NaN,C:\coding\wgscraper\data\ronorp\adpage_2020-10-19_00-09-27_570444.scrape,https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wunderschoenes-co-living-haus-mit-38-suiten-in-kloten.2042131?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1,False,2020-10-19 00:09:27.570444
NaN,2,2.0,https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/schoenes-studio-in-wollerau.2072370?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1%26sort%3Dcreated%26page%3D7%26hash%3D8066eefae06f4ebdde76511cbb811cdf,Schönes Studio in Wollerau,[],click,2072370_1220_3bd7300806c559e9a9c92bb64096ff30,https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/schoenes-studio-in-wollerau.2072370?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1%26sort%3Dcreated%26page%3D7%26hash%3D8066eefae06f4ebdde76511cbb811cdf,ContentList,ContentTitle,NaN,C:\coding\wgscraper\data\ronorp\adpage_2020-10-19_00-09-59_659456.scrape,https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/schoenes-studio-in-wollerau.2072370?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1%26sort%3Dcreated%26page%3D7%26hash%3D8066eefae06f4ebdde76511cbb811cdf,False,2020-10-19 00:09:59.659456
NaN,3,3.0,https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/3-5-zimmer-luxuswohnung-zu-vermieten-in-bilten.2072351?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1%26sort%3Dcreated%26page%3D7%26hash%3D8066eefae06f4ebdde76511cbb811cdf,"3,5 Zimmer Luxuswohnung zu vermieten in Bilten",[],click,2072351_1220_bb8d551ebe16be76d4bdb276fad4c24a,https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/3-5-zimmer-luxuswohnung-zu-vermieten-in-bilten.2072351?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1%26sort%3Dcreated%26page%3D7%26hash%3D8066eefae06f4ebdde76511cbb811cdf,ContentList,ContentTitle,NaN,C:\coding\wgscraper\data\ronorp\adpage_2020-10-19_00-10-05_825444.scrape,https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/3-5-zimmer-luxuswohnung-zu-vermieten-in-bilten.2072351?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1%26sort%3Dcreated%26page%3D7%26hash%3D8066eefae06f4ebdde76511cbb811cdf,False,2020-10-19 00:10:05.825444
NaN,4,4.0,https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/dachzimmer-in-wg-am-brupbacherplatz-wochenaufenthalter-in.2070785?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1%26page%3D13%26hash%3D8066eefae06f4ebdde76511cbb811cdf,Dachzimmer in WG am Brupbacherplatz //...,[],click,2070785_1220_55792fd7a5cf63254d44f7a2

In [66]:
a=pd.DataFrame()
a={'a':{'nr1':1,'nr2':2}}
a.update({'b':{'nr1':3,'nr2':4}})
b=pd.DataFrame(a).transpose()
b

,nr1,nr2
a,1,2
b,3,4


In [33]:
import datetime

In [43]:
datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S_%f")

'2020-10-18_16-53-42_257201'

In [69]:
from pathlib import Path
a={'https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wunderschoenes-co-living-haus-mit-38-suiten-in-kloten.2042131?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1': {'data-advert-position': '2042131_1220_20941f96bcb24379f597ac78d9fc0d78',
  'class': [],
  'href': 'https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wunderschoenes-co-living-haus-mit-38-suiten-in-kloten.2042131?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1',
  'data-attr-href': 'https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wunderschoenes-co-living-haus-mit-38-suiten-in-kloten.2042131?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1',
  'alt': 'Wunderschönes Co-Living Haus mit 38 Suiten in Kloten,...',
  'data-trace': '',
  'data-category': 'ContentList',
  'data-label': 'ContentTitle',
  'data-actions': 'click',
  'fname': 'C:/coding/wgscraper/data/ronorp/adpage_2020-10-18_22-08-50_909156.scrape',
  'scrape_ts': datetime.datetime(2020, 10, 18, 22, 8, 50, 909156),
  'processed': False},
 'https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wg-zimmer-k5-befristet-fuer-6-monate-eventuell-auch-fuer.2073431?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1': {'data-advert-position': '2073431_1220_995118c3e235ede9f70e283eaac961d1',
  'class': [],
  'href': 'https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wg-zimmer-k5-befristet-fuer-6-monate-eventuell-auch-fuer.2073431?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1',
  'data-attr-href': 'https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wg-zimmer-k5-befristet-fuer-6-monate-eventuell-auch-fuer.2073431?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1',
  'alt': 'WG zimmer K5 befristet für 6 Monate (eventuell auch fuer...',
  'data-trace': '',
  'data-category': 'ContentList',
  'data-label': 'ContentTitle',
  'data-actions': 'click',
  'fname': 'C:/coding/wgscraper/data/ronorp/adpage_2020-10-18_22-08-55_236071.scrape',
  'scrape_ts': datetime.datetime(2020, 10, 18, 22, 8, 55, 236071),
  'processed': False},
 'https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/mitbewohnerin-gesucht-ab-1-nov-dez-od-nach-vereinbarung.2073422?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1%26page%3D2%26hash%3D8066eefae06f4ebdde76511cbb811cdf': {'data-advert-position': '2073422_1220_ee0e5471d1e37033c2d82a73050ad0b0',
  'class': [],
  'href': 'https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/mitbewohnerin-gesucht-ab-1-nov-dez-od-nach-vereinbarung.2073422?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1%26page%3D2%26hash%3D8066eefae06f4ebdde76511cbb811cdf',
  'data-attr-href': 'https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/mitbewohnerin-gesucht-ab-1-nov-dez-od-nach-vereinbarung.2073422?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1%26page%3D2%26hash%3D8066eefae06f4ebdde76511cbb811cdf',
  'alt': 'Mitbewohnerin gesucht ab 1. Nov/Dez od. nach Vereinbarung',
  'data-trace': '',
  'data-category': 'ContentList',
  'data-label': 'ContentTitle',
  'data-actions': 'click',
  'fname': 'C:/coding/wgscraper/data/ronorp/adpage_2020-10-18_22-08-58_837035.scrape',
  'scrape_ts': datetime.datetime(2020, 10, 18, 22, 8, 58, 837035),
  'processed': False},
 'https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wg-zimmer-in-4er-wg-in-5-min-gehdistanz-zum-hb.2073417?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1%26page%3D2%26hash%3D8066eefae06f4ebdde76511cbb811cdf': {'data-advert-position': '2073417_1220_4ace612802ebbbfe74b3f1383a14378a',
  'class': [],
  'href': 'https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wg-zimmer-in-4er-wg-in-5-min-gehdistanz-zum-hb.2073417?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1%26page%3D2%26hash%3D8066eefae06f4ebdde76511cbb811cdf',
  'data-attr-href': 'https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wg-zimmer-in-4er-wg-in-5-min-gehdistanz-zum-hb.2073417?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1%26page%3D2%26hash%3D8066eefae06f4ebdde76511cbb811cdf',
  'alt': 'WG-Zimmer in 4er-WG in 5 min. Gehdistanz zum HB',
  'data-trace': '',
  'data-category': 'ContentList',
  'data-label': 'ContentTitle',
  'data-actions': 'click',
  'fname': 'C:/coding/wgscraper/data/ronorp/adpage_2020-10-18_22-09-02_609034.scrape',
  'scrape_ts': datetime.datetime(2020, 10, 18, 22, 9, 2, 609034),
  'processed': False},
 'https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wunderschoenes-zimmer-mit-eigenem-bad-im-5-stock-in-toplage.2073410?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1%26page%3D2%26hash%3D8066eefae06f4ebdde76511cbb811cdf': {'data-advert-position': '2073410_1220_11278ecccec3fde69f6b169b9aa10632',
  'class': [],
  'href': 'https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wunderschoenes-zimmer-mit-eigenem-bad-im-5-stock-in-toplage.2073410?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1%26page%3D2%26hash%3D8066eefae06f4ebdde76511cbb811cdf',
  'data-attr-href': 'https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wunderschoenes-zimmer-mit-eigenem-bad-im-5-stock-in-toplage.2073410?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1%26page%3D2%26hash%3D8066eefae06f4ebdde76511cbb811cdf',
  'alt': 'Wunderschönes Zimmer mit eigenem Bad im 5. Stock in...',
  'data-trace': '',
  'data-category': 'ContentList',
  'data-label': 'ContentTitle',
  'data-actions': 'click',
  'fname':'C:/coding/wgscraper/data/ronorp/adpage_2020-10-18_22-09-05_941036.scrape',
  'scrape_ts': datetime.datetime(2020, 10, 18, 22, 9, 5, 941036),
  'processed': False}}


In [82]:
b=pd.DataFrame(a).transpose()
b.index.name = 'link'
b

,data-advert-position,class,href,data-attr-href,alt,data-trace,data-category,data-label,data-actions,fname,scrape_ts,processed
link,,,,,,,,,,,,
https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wunderschoenes-co-living-haus-mit-38-suiten-in-kloten.2042131?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1,2042131_1220_20941f96bcb24379f597ac78d9fc0d78,[],https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wunderschoenes-co-living-haus-mit-38-suiten-in-kloten.2042131?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1,https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wunderschoenes-co-living-haus-mit-38-suiten-in-kloten.2042131?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1,"Wunderschönes Co-Living Haus mit 38 Suiten in Kloten,...",,ContentList,ContentTitle,click,C:/coding/wgscraper/data/ronorp/adpage_2020-10-18_22-08-50_909156.scrape,2020-10-18 22:08:50.909156,False
https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wg-zimmer-k5-befristet-fuer-6-monate-eventuell-auch-fuer.2073431?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1,2073431_1220_995118c3e235ede9f70e283eaac961d1,[],https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wg-zimmer-k5-befristet-fuer-6-monate-eventuell-auch-fuer.2073431?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1,https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wg-zimmer-k5-befristet-fuer-6-monate-eventuell-auch-fuer.2073431?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1,WG zimmer K5 befristet für 6 Monate (eventuell auch fuer...,,ContentList,ContentTitle,click,C:/coding/wgscraper/data/ronorp/adpage_2020-10-18_22-08-55_236071.scrape,2020-10-18 22:08:55.236071,False
https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/mitbewohnerin-gesucht-ab-1-nov-dez-od-nach-vereinbarung.2073422?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1%26page%3D2%26hash%3D8066eefae06f4ebdde76511cbb811cdf,2073422_1220_ee0e5471d1e37033c2d82a73050ad0b0,[],https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/mitbewohnerin-gesucht-ab-1-nov-dez-od-nach-vereinbarung.2073422?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1%26page%3D2%26hash%3D8066eefae06f4ebdde76511cbb811cdf,https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/mitbewohnerin-gesucht-ab-1-nov-dez-od-nach-vereinbarung.2073422?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1%26page%3D2%26hash%3D8066eefae06f4ebdde76511cbb811cdf,Mitbewohnerin gesucht ab 1. Nov/Dez od. nach Vereinbarung,,ContentList,ContentTitle,click,C:/coding/wgscraper/data/ronorp/adpage_2020-10-18_22-08-58_837035.scrape,2020-10-18 22:08:58.837035,False
https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wg-zimmer-in-4er-wg-in-5-min-gehdistanz-zum-hb.2073417?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1%26page%3D2%26hash%3D8066eefae06f4ebdde76511cbb811cdf,2073417_1220_4ace612802ebbbfe74b3f1383a14378a,[],https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wg-zimmer-in-4er-wg-in-5-min-gehdistanz-zum-hb.2073417?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1%26page%3D2%26hash%3D8066eefae06f4ebdde76511cbb811cdf,https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wg-zimmer-in-4er-wg-in-5-min-gehdistanz-zum-hb.2073417?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1%26page%3D2%26hash%3D8066eefae06f4ebdde76511cbb811cdf,WG-Zimmer in 4er-WG in 5 min. Gehdistanz zum HB,,ContentList,ContentTitle,click,C:/coding/wgscraper/data/ronorp/adpage_2020-10-18_22-09-02_609034.scrape,2020-10-18 22:09:02.609034,False
https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wunderschoenes-zimmer-mit-eigenem-bad-im-5-stock-in-toplage.2073410?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1%26page%3D2%26hash%3D8066eefae06f4ebdde76511cbb811cdf,2073410_1220_11278ecccec3fde69f6b169b9aa10632,[],https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wunderschoenes-zimmer-mit-eigenem-bad-im-5-stock-in-toplage.20734

In [84]:
b['href']

link
https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wunderschoenes-co-living-haus-mit-38-suiten-in-kloten.2042131?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1                                                                                                                          https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wunderschoenes-co-living-haus-mit-38-suiten-in-kloten.2042131?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1
https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wg-zimmer-k5-befristet-fuer-6-monate-eventuell-auch-fuer.2073431?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1                                                                                                                    https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wg-zimmer-k5-befristet-fuer-6-monate-eventuell-auch-fuer.2073431?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1
https://www.ronorp.net/zuerich/im